[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Machine Learning - Deep Learning - Object Detection (Satellite Object Detection)

Working on data from [MAFAT Challenge - Satellite Vision Challenge](https://codalab.lisn.upsaclay.fr/competitions/9603).

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 14/07/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0099DeepLearningObjectDetection.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Deep Learning

# Image Processing and Computer Vision
import skimage as ski

# Miscellaneous
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import shutil

# Typing
from typing import Callable, Dict, Generator, List, Literal, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

TEST_FOLDER_NAME  = 'Test'
TRAIN_FOLDER_NAME = 'Train'
VAL_FOLDER_NAME   = 'Validation'

In [ ]:
# Download Auxiliary Modules for Google Colab


In [ ]:
# Courses Packages

from AuxFun import BBoxFormat
from AuxFun import ConvertBBoxFormat

In [ ]:
# General Auxiliary Functions


In [ ]:
# Auxiliary Functions

def ParsePolygonFile(filePath: str) -> Optional[List[Tuple[List[float], str]]]:
    """
    Parses a polygon annotation file where each line has the format:
        x1 y1 x2 y2 x3 y3 x4 y4 className

    Parameters
    ----------
    filePath : str
        Path to the annotation text file.

    Returns
    -------
    List[Tuple[List[float], str]] or None
        A list where each item is a tuple:
            ([x1, y1, x2, y2, x3, y3, x4, y4], className)
        or None if the file is empty.
    """
    parsed_data = []

    with open(filePath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            parts = line.split()
            if len(parts) < 9:
                raise ValueError(f"Invalid format: line does not contain 8 coordinates and a class name: {line}")

            coords = list(map(float, parts[:8]))
            className = parts[8]
            parsed_data.append((coords, className))

    return parsed_data if parsed_data else None


In [ ]:
# Parameters 

tuImgSize = (1280, 1280)

dClass = {
    'small_vehicle': 0,
    'medium_vehicle': 1,
    'large_vehicle': 2,
    'bus': 3,
    'double_trailer_truck': 4,
    'container': 5,
    'heavy_equipment': 6,
    'pylon': 7,
    'small_aircraft': 8,
    'large_aircraft': 9,
    'small_vessel': 10,
    'medium_vessel': 11,
    'large_vessel': 12,
}

dataFileUrl = r'https://technionmail-my.sharepoint.com/:u:/g/personal/royia_technion_ac_il/EQekWAqWVFdEkiKUW1L6MzcB3Cw0dxYazr0uJvuv4tFM3g?e=WzOjN2' #<! Course OneDrive

In [ ]:
# Paths

dataFolderPath = os.path.join('Data')
imgFolderPath = os.path.join(dataFolderPath, 'RAW', 'Images')
lblFolderPath = os.path.join(dataFolderPath, 'RAW', 'Labels')

yoloImagesFolderPath = os.path.join(dataFolderPath, 'images')
yoloLabelsFolderPath = os.path.join(dataFolderPath, 'labels')

In [ ]:
# Image Files
lFiles        = os.listdir(imgFolderPath)
lImgFiles     = [fileName for fileName in lFiles if fileName.endswith('tiff')]

lImgFilesBaseName = [os.path.splitext(fileName)[0] for fileName in lImgFiles]
lImgFilesBaseName

In [ ]:
lFiles    = os.listdir(lblFolderPath)
lLblFiles = [fileName for fileName in lFiles if fileName.endswith('txt')]
lLblFilesBaseName = [os.path.splitext(fileName)[0] for fileName in lLblFiles]
lLblFilesBaseName

In [ ]:
# Set Intersection

dImgFiles = set(lImgFilesBaseName)
dLblFiles = set(lLblFilesBaseName)

dLblFiles == dImgFiles

In [ ]:
# Sort to match order
lImgFiles.sort()
lLblFiles.sort()

In [ ]:
# Folders

yoloImagesFolderPath = os.path.join(dataFolderPath, 'images')
yoloLabelsFolderPath = os.path.join(dataFolderPath, 'labels')

# Clean Images folder
if os.path.isdir(yoloImagesFolderPath):
    shutil.rmtree(yoloImagesFolderPath) 

# Clean Labels folder
if os.path.isdir(yoloLabelsFolderPath):
    shutil.rmtree(yoloLabelsFolderPath) 

os.makedirs(os.path.join(yoloImagesFolderPath, TRAIN_FOLDER_NAME))
os.makedirs(os.path.join(yoloImagesFolderPath, VAL_FOLDER_NAME))
os.makedirs(os.path.join(yoloLabelsFolderPath, TRAIN_FOLDER_NAME))
os.makedirs(os.path.join(yoloLabelsFolderPath, VAL_FOLDER_NAME))

In [ ]:
# Generate Labels

lValidImg = []

for ii, fileName in enumerate(lLblFiles):
    filePath = os.path.join(lblFolderPath, fileName)
    lFileData = ParsePolygonFile(filePath)
    if lFileData is None:
        lValidImg.append(False)
        continue

    lValidImg.append(True)
    with open(os.path.join(yoloLabelsFolderPath, fileName), 'w') as hFile:

        for tuPolygon in lFileData:
            lCoord = tuPolygon[0] #<! (x1, y1, x2, y2, x3, y3, x4, y4)
            xMin = np.min(lCoord[0::2])
            xMax = np.max(lCoord[0::2])
            yMin = np.min(lCoord[1::2])
            yMax = np.max(lCoord[1::2])

            vBox = np.array([xMin, yMin, xMax, yMax])

            vB = ConvertBBoxFormat(vBox, tuImgSize, BBoxFormat.PASCAL_VOC, BBoxFormat.YOLO)

            lineStr = f'{dClass[tuPolygon[1]]} {vB[0]:0.3f} {vB[1]:0.3f} {vB[2]:0.3f} {vB[3]:0.3f}\n'

            hFile.write(lineStr)

In [ ]:
# Generate Images

for ii, fileName in enumerate(lImgFiles):
    print(f'Processing image #{(ii + 1):004} / {len(lImgFiles)}')
    if not lValidImg[ii]:
        continue
    filePath = os.path.join(imgFolderPath, fileName)
    fileNameBase = os.path.splitext(fileName)[0]
    
    mI = ski.io.imread(filePath)
    mI = ski.util.img_as_float64(mI)
    if np.ndim(mI) == 2:
        mI = mI[:, :, None]
    if np.size(mI, 2) == 1:
        mI = np.tile(mI, (1, 1, 3))
    mI = ski.exposure.rescale_intensity(mI)
    mI = ski.util.img_as_ubyte(mI)

    ski.io.imsave(os.path.join(yoloImagesFolderPath, fileNameBase + '.png'), mI)